# Algorithme de base de région de confiance

In [ ]:
using LinearAlgebra
# using Optim

Nous allons définir une structure simple permettant de conserver les paramètres de l'algorithmes.

In [ ]:
struct BasicTrustRegion{T <: Real}
    η1:: T
    η2:: T
    γ1:: T
    γ2:: T
end

function BTRDefaults()
    # Nous utilisons les valeurs par défaut définies dans le livres
    # "Trust-region methods" (2000)
    return BasicTrustRegion(0.01,0.9,0.5,0.5)
end

L'état de l'algorithme est déclaré avec le mot clé addtionnel `mutable` comme le contenu peut être modifié d'itération en itération.

In [ ]:
mutable struct BTRState
    iter::Int
    x::Vector
    xcand::Vector
    g::Vector
    step::Vector
    Δ::Float64
    ρ::Float64
    
    function BTRState()
        return new()
    end
end

La fonction suivante permet de décider si l'itéré candidat est accepté ou refusé.

In [ ]:
function acceptCandidate!(state::BTRState, b::BasicTrustRegion)
    # If the iteration is successful, update the iterate
    if (state.ρ >= b.η1)
        state.x = copy(state.xcand) # x[:] = state.xcand
        return true
    else
        return false
    end
end

Nous définissons à présent la méthode de mise à jour du rayon de la région de confiance.

In [ ]:
function updateRadius!(state::BTRState, b::BasicTrustRegion)
    if (state.ρ >= b.η2)
        # very successful iterate
        state.Δ = min(1e20,max(4*norm(state.step),state.Δ))
    elseif (state.ρ >= b.η1)
        # successful iterate
        state.Δ *= b.γ2
    else
        # unsuccessful iterate
        state.Δ *= b.γ1
    end
end

## Pas de Cauchy

Calculons et vérifions la valeur du pas de Cauchy.

On sait que
$$
\alpha^* =
\begin{cases}
\min \left\{ \frac{\Delta_k}{\| \nabla f(x_k) \|_k}, \frac{\| \nabla f(x_k) \|_2^2}{\nabla f(x_k)^T H_k \nabla f(x_k)}  \right\} & \text{si } \nabla f(x_k)^T H_k \nabla f(x_k) > 0 \\
\frac{\Delta_k}{\| \nabla f(x_k) \|_k} & \text{sinon.}
\end{cases}
$$
Le pas de Cauchy est alors
$$
s_k = -\alpha^*\nabla f(x^*).
$$
Dans la suite, nous utiliserons la norme euclidienne, i.e. $\| \cdot \|_k = \| \cdot \|_2$.

Soit
$$
q = \nabla f(x_k)^T H_k \nabla f(x_k)
$$
Si $q \leq 0$, nous sommes dans une direction de courbure non positive, et nous devons nous devons rendre à la frontière. Dès lors
$$
s_k = - \frac{\Delta_k}{\| \nabla f(x_k) \|_k} \nabla f(x_k).
$$

Si $q > 0$, nous devons vérifier si le minimum du modèle le long de la plus forte pente est à l'intérieur de la région de confiance. Si c'est le cas, nous avons
\begin{align*}
s_k &= - \frac{\| \nabla f(x_k) \|_2^2}{\nabla f(x_k)^T H_k \nabla f(x_k)} \nabla f(x_k) \\
&= - \frac{\| \nabla f(x_k) \|_2^2}{\nabla f(x_k)^T H_k \nabla f(x_k)} \nabla f(x_k) \frac{\Delta_k}{\| \nabla f(x_k) \|_k}
\frac{\| \nabla f(x_k) \|_k}{\Delta_k} \\
&= - \frac{\| \nabla f(x_k) \|_2^3}{\Delta_k \nabla f(x_k)^T H_k \nabla f(x_k)} \frac{\Delta_k}{\| \nabla f(x_k) \|_k} \nabla f(x_k) \\
&= - \tau \frac{\Delta_k}{\| \nabla f(x_k) \|_k} \nabla f(x_k),
\end{align*}
où
$$
\tau = \frac{\| \nabla f(x_k) \|_2^3}{\Delta_k q}.
$$

Si ce n'est pas le cas, nous nous arrêtons sur la frontière de la région de confiance, et nous avons à nouveau
$$
s_k = - \frac{\Delta_k}{\| \nabla f(x_k) \|_k} \nabla f(x_k).
$$

En regroupant ces deux cas, nous avons
$$
s_k = - \tau \frac{\Delta_k}{\| \nabla f(x_k) \|_k} \nabla f(x_k),
$$
avec
$$
\tau = \min \left\{ 1.0, \frac{\| \nabla f(x_k) \|_2^3}{\Delta_k q} \right\}.
$$
Ceci permet de construire le code ci-après.

In [ ]:
function CauchyStep(g::Vector,H::Matrix,Δ::Float64)
    q = dot(g,H*g)
    normg = norm(g)

    if (q <= 0)
        # the curvature along g is non positive
        τ = 1.0
    else
        # the curvature along g is positive
        τ = min((normg*normg*normg)/(q*Δ),1.0)
    end

    return -τ*g*Δ/normg # return the step
end

## Algorithme de base de région de confiance

Nous sommes à présent en mesure de proposer une première implémentation de la méthode de région de confiance.

In [ ]:
function btr(f::Function, g!::Function, H!::Function, x0::Vector;
             tol::Float64 = 1e-6, verbose::Bool = false)
    
    b = BTRDefaults()
    state = BTRState()
    state.iter = 0
    state.x = x0
    n=length(x0)

    state.g = zeros(n)
    H = zeros(n,n)
    
    fx = f(x0)
    g!(state.g, x0)
    H!(H, x0)

    init_radius!(state)
    
    nmax = 100000
    
    tol2 = tol*tol
    
    function model(s::Vector, g::Vector, H::Matrix)
        return dot(s, g)+0.5*dot(s, H*s)
    end
    
    if (verbose)
        println(state)
    end

    while (dot(state.g,state.g) > tol2 && state.iter < nmax)
        # Compute the step by approximately minimize the model
        state.step = CauchyStep(state.g, H, state.Δ)
        state.xcand = state.x+state.step

        # Compute the actual reduction over the predicted reduction
        fcand = f(state.xcand)
        state.ρ = (fcand-fx)/(model(state.step, state.g, H))

        if (acceptCandidate!(state, b))
            g!(state.g, state.x)
            H!(H, state.x)
            fx = fcand
        end

        if (verbose)
            println(state)
        end
        
        updateRadius!(state, b)
        state.iter += 1
    end
    
    return state
end

Dans un premier temps, nous allons initialiser le rayon de la région de confiance à 1.

In [ ]:
function init_radius!(state:: BTRState)
    state.Δ = 1.0
end

## Exemple

### Fonction de Rosenbrock

Vu l'importance de la fonction de Rosenbrock à des fins de tests, nous l'incluons dans du code séparé que nous incorporons au bloc-notes avec la commande `include`.

In [ ]:
include("rosenbrock.jl");

Nous pouvons interroger la documentation associée comme suit.

In [ ]:
? rosenbrock

Nous pouvons tester notre première implémentation sur Rosenbrock.

In [ ]:
state = btr(rosenbrock, rosenbrock_gradient!, rosenbrock_hessian!, [0,0])

Regardons quel est le rayon final de la région de confiance.

In [ ]:
state.Δ

Vérifions la première itération, en partant de (0,0). Afin de calculer la longueur du pas, nous devons d'abord connaître le gradient et la matrice hessienne en (0,0).

In [ ]:
x = [0; 0]
grad=zeros(2)
hess=zeros(2,2)
rosenbrock_gradient!(grad,x)
rosenbrock_hessian!(hess,x)

Nous savons que
$$
\alpha^* = \min
\left\{
\frac{\Delta_k}{\| \nabla f(x_k) \|_k},
\frac{\| \nabla f(x_k) \|_2^2}{\nabla f(x_k)^T H_k \nabla f(x_k)}
\right\}
$$
Le premier terme de la minimisation est

In [ ]:
1.0/norm(grad)

In [ ]:
grad

Le second est

In [ ]:
dot(grad,grad)/dot(grad,hess*grad)

Dès lors $\alpha^* = 0.5$ et $x_1 = (0,0) + 0.5*\nabla f(0.0) = (1,0)$. Nous pouvons facilement vérifier cette valeur en regardant les premières itérations de la procédure d'optimisation.

In [ ]:
state = btr(rosenbrock, rosenbrock_gradient!, rosenbrock_hessian!, [0,0], verbose = true)

### Exemple 2

Considérons à la présent la fonction
$$
f(x) = x_1^4 + x_1^2 + x_1*x_2 + (1+x_2)^2,
$$
dont nous calculons le gradient et la matrice hessienne par différentiation automatique.

In [ ]:
using ForwardDiff

f(x) = x[1]^4 + x[1]^2 + x[1]*x[2] + (1+x[2])^2
g = x -> ForwardDiff.gradient(f, x);
H = x -> ForwardDiff.hessian(f, x);
function g!(storage::Vector, x::Vector)
    s = g(x)
    storage[1:length(s)] = s[1:length(s)]
end
function H!(storage::Matrix, x::Vector)
    s = H(x)
    n, m = size(s)
    storage[1:n,1:m] = s[1:length(s)]
end

L'application de la méthode de région de confiance donne:

In [ ]:
state = btr(f, g!, H!, [0,0], verbose = true)

Nous pouvons constater que le nombre d'itérations est nettement plus raisonnable qu'avec Rosenbrock.

## Initialisation du rayon de la région de confiance

Le code précédent défini arbitrairement le rayon initial de la région de confiance avec la valeur 1. Cette valeur peut se révéler trop petite, et un grand nombre d'itérations sont nécessaires avant d'obtenir un voisinage assez grand, ou trop grand, conduisant à de nombreuses itérations non réussies en début d'exécution.

Plusieurs approches ont été proposées dans la littérature. Une heuristique populaire, proposée initialement dans le logiciel `Lancelot`, est de définir
$$
\Delta_0 = \kappa \| \nabla f(x_0) \|,
$$
où $x_0$ est le point de départ et $\kappa > 0$. On pourra prendre $\kappa = 0.1$.

In [ ]:
function init_radius!(state:: BTRState)
    state.Δ = 0.1*norm(state.g)
end

Essayons sur Rosenbrock.

In [ ]:
state = btr(rosenbrock, rosenbrock_gradient!, rosenbrock_hessian!, [0,0])

C'est nettement mieux!

Regardons sur le deuxième exemple.

In [ ]:
state = btr(f, g!, H!, [0,0], verbose = true)

On voit que ce n'est pas toujours efficace!